###### Analysis of glyco-phenotypes annotations in diseases

Background: Given a set of diseases that should have at least one glycophenotype annotation, explore the HPO disease to phenotype annotation dataset to see if they exist.

Requires:
- rdflib
- prefixcommons

In [28]:
from rdflib import URIRef, BNode, Literal, Graph, RDFS
from prefixcommons import contract_uri, expand_uri
from typing import Set, Optional
import csv
import gzip


# Open file with disease list
diseases = []
with open("./input/glyco-mondo.txt", 'r') as glyco_file:
    for line in glyco_file:
        diseases.append(line.rstrip("\n"))

d2p = {}
with open('../data/mondo_hp_w_groups.tsv', 'r') as cache_file:
    reader = csv.reader(cache_file, delimiter='\t', quotechar='\"')
    for row in reader:
        if row[0].startswith('#'): continue
        (mondo_id, phenotype_id) = row[0:2]
        if mondo_id in diseases:
            if mondo_id in d2p:
                d2p[mondo_id].add(phenotype_id)
            else:
                d2p[mondo_id] = {phenotype_id}

# Function for getting reflexive subclass_of closure
# from rdflib Graph
def get_subclass_closure(
        graph: Graph,
        node: str,
        root: Optional[str] = None) -> Set[str]:
    nodes = set()
    root_seen = {}
    node = URIRef(expand_uri(node, strict=True))

    if root is not None:
        root = URIRef(expand_uri(root, strict=True))
        root_seen = {root: 1}
    for obj in graph.transitive_objects(node, RDFS['subClassOf'], root_seen):
        if isinstance(obj, Literal) or isinstance(obj, BNode):
            continue
        nodes.add(contract_uri(str(obj), strict=True)[0])

    # Add root to graph
    if root is not None:
        nodes.add(contract_uri(str(root), strict=True)[0])

    return nodes

print("Diseases to process: {}".format(len(diseases)))

Diseases to process: 185


In [29]:
# Set of glyco-phenotype grouping classes
# TODO unsure if this is definitive list
glyco_phenogroups = {
    'HP:0011014', # Abnormal glucose homeostasis
    'HP:0012345', # Abnormal glycosylation
    'HP:0004366', # Abnormality of glycolysis
    'HP:0003649', # Abnormality of glycoside metabolism
    'HP:0011012', # Abnormality of polysaccharide metabolism
    'HP:0012067', # Glycopeptiduria
    'HP:0010471'  # Oligosacchariduria
}

# Load hpo owl file into memory, this takes ~ 1-2 minutes
hpo = Graph()
hpo.parse(gzip.open("../data/hp.owl.gz", 'rb'), format='xml')
root = "HP:0000118"

In [32]:
# Output file
output = open('./output/glyco-mondo-annotations.tsv', 'w')

headers = [
    'mondo_id',           # MONDO curie
    'num_glycophen',      # Count of glycophenotypes annotated to disease
    'num_nonglyco',       # Count of non-glycophentypes annotated to disease
    'glycophenotypes',    # Pipe delim list of glycophenotype terms
    'non_glycophenotypes' # Pipe delim list of non-glycophenotype terms
]

output.write("{}\n".format("\t".join(headers)))


# Iterate over diseases and write to file
for disease in diseases:
    glyco_count = 0
    nonglyco_count = 0
    glycophenotypes = []
    non_glycophenotypes = []
    if disease not in d2p:
        print("debug: {} not in annotation file".format(disease))
        continue
    for phenotype in d2p[disease]:
        phenotype_closure = get_subclass_closure(hpo, phenotype, root)
        if len(phenotype_closure.intersection(glyco_phenogroups)) > 0:
            glyco_count += 1
            glycophenotypes.append(phenotype)
        else:
            nonglyco_count += 1
            non_glycophenotypes.append(phenotype)
            
    # Write line to file
    output.write("{}\t{}\t{}\t{}\t{}\n".format(
        disease,
        glyco_count,
        nonglyco_count,
        "|".join(glycophenotypes),
        "|".join(non_glycophenotypes),
    ))

debug: MONDO:0000182 not in annotation file
debug: MONDO:0000189 not in annotation file
debug: MONDO:0011142 not in annotation file
debug: MONDO:0013303 not in annotation file
debug: MONDO:0017734 not in annotation file
debug: MONDO:0018163 not in annotation file
debug: MONDO:0018273 not in annotation file
debug: MONDO:0018274 not in annotation file
debug: MONDO:0018485 not in annotation file
debug: MONDO:0029135 not in annotation file
debug: MONDO:0060640 not in annotation file
